## Data Loading and splitting

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import tensorflow as tf

data = fetch_california_housing()
X, y = data.data, data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import pandas as pd
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = data.target
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## RBF Class

In [ ]:
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, units, gamma=0.1):
        super(RBFLayer, self).__init__()
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(self.units, input_shape[-1]),
                                       initializer='random_uniform',
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.units,),
                                     initializer=tf.keras.initializers.Constant(1.0),
                                     trainable=True)

    def call(self, inputs):
        C = tf.expand_dims(self.centers, axis=0)
        X = tf.expand_dims(inputs, axis=1)
        distances = tf.reduce_sum((X - C) ** 2, axis=-1)
        return tf.exp(-self.gamma * distances)

rbf_model = Sequential([
    Input(shape=(X.shape[1],)),
    RBFLayer(units=10, gamma=0.1),
    Dense(1)
])


In [ ]:
rbf_model.compile(optimizer=Adam(), loss=MeanSquaredError())

In [ ]:
print("Checking for NaN in target data:")
print(f"NaN in y_train: {np.isnan(y_train).sum()}, NaN in y_test: {np.isnan(y_test).sum()}")

Checking for NaN in target data:
NaN in y_train: 0, NaN in y_test: 0


### RBF Testing

In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['betas'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


RBF Model Loss: 0.4538183808326721


In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...
RBF Model Loss: 0.413193017244339


In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...
RBF Model Loss: 0.4164479672908783


In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...
RBF Model Loss: 0.4073345959186554


In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=200, batch_size=64, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...
RBF Model Loss: 0.3986707031726837


In [ ]:
print("Training RBF Model...")
rbf_model.fit(X_train, y_train, epochs=300, batch_size=64, verbose=0)

rbf_loss = rbf_model.evaluate(X_test, y_test, verbose=0)
print(f"RBF Model Loss: {rbf_loss}")

Training RBF Model...
RBF Model Loss: 0.3910798132419586


## Dense Model

In [ ]:
simple_model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

simple_model.compile(optimizer=Adam(), loss=MeanSquaredError())

### Dense Testing

In [ ]:
print("Training Dense Model...")
simple_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

dense_loss = simple_model.evaluate(X_test, y_test, verbose=0)
print(f"Dense Model Loss: {dense_loss}")

Training Dense Model...
Dense Model Loss: 0.2660219073295593


In [ ]:
print("Training Dense Model...")
simple_model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=0)

dense_loss = simple_model.evaluate(X_test, y_test, verbose=0)
print(f"Dense Model Loss: {dense_loss}")

Training Dense Model...
Dense Model Loss: 0.2720809876918793


In [ ]:
print("Training Dense Model...")
simple_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

dense_loss = simple_model.evaluate(X_test, y_test, verbose=0)
print(f"Dense Model Loss: {dense_loss}")

Training Dense Model...
Dense Model Loss: 0.2608625292778015


In [ ]:
print("Training Dense Model...")
simple_model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)

dense_loss = simple_model.evaluate(X_test, y_test, verbose=0)
print(f"Dense Model Loss: {dense_loss}")

Training Dense Model...
Dense Model Loss: 0.2644297480583191
